In [9]:
import os
import sys
import numpy as np
import keras.callbacks as cb
import keras.utils.np_utils as np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.core import Activation
from keras import applications # For easy loading the VGG_16 Model
from skimage import color
import cv2
# Image loading and other helper functions
import dwdii_bc_model_helper as bc

## Image_Augmentation
The following function takes the 8bit grayscale images that we are using and performs a series of affine transformations to the images. There are vertical and horizontal flips along with rotations of 90, 270, 15, 30, and 45 degrees. Also included is a function for generating the rotations. Image augmentation needs to be performed before runnin the VGG_Prep function.  

When calling the Image_Augmentation function setting the various flags to True will cause the transformation to be performed.

In [4]:
# Function for rotating the image files.
def Image_Rotate(img, angle):
    """
    Rotates a given image the requested angle. Returns the rotated image.
    """
    rows,cols = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), angle, 1)
    return(cv2.warpAffine(img,M,(cols,rows)))

# Function for augmenting the images
def Image_Augment(X, Y, vflip=False, hflip=False, major_rotate=False, minor_rotate=False):
    """
    :param  X np.array of images
            Y np.array of labels
            vflip, hflip, major_rotate, minor_rotate set to True to perform the augmentations
    :return The set of augmented iages and their corresponding labels
    
    """
    if len(X) != len(Y):
        print('Data and Label arrays not of the same length.')
    
    n = vflip + hflip + 2*major_rotate + 6*minor_rotate
    augmented = np.zeros([len(X) + n*len(X), X.shape[1], X.shape[2]])
    label = np.zeros([len(Y) + n*len(Y), 1])
    count = 0
    for i in range(0, len(X)):
        augmented[count] = X[i]
        label[count] = Y[i]
        count += 1
        if vflip:
            aug = cv2.flip(X[i], 0)
            augmented[count] = aug
            label[count] = Y[i]
            count += 1
        if hflip:
            aug = cv2.flip(X[i], 1)
            augmented[count] = aug
            label[count] = Y[i]
            count +=1 
        if major_rotate:
            angles = [90, 270]
            for angle in angles:
                aug = Image_Rotate(X[i], angle)
                augmented[count] = aug
                label[count] = Y[i]
                count += 1
        if minor_rotate:
            angles = [-45,-30,-15,15,30,45]
            for angle in angles:
                aug = Image_Rotate(X[i], angle)
                augmented[count] = aug
                label[count] = Y[i]
                count += 1
                
    return(augmented, label)

### VGG_Prep
The following function takes the 8bit grayscale images that we are using and converts them to 8bit rgb while at the same time changing the pixles to a scale of 0 to 255. These image parameters are required by the VGG_16 model. 

In [5]:
def VGG_Prep(img_data):
    """
    :param img_data: training or test images of shape [#images, height, width]
    :return: the array transformed to the correct shape for the VGG network
                shape = [#images, height, width, 3] transforms to rgb and reshapes
    """
    images = np.zeros([len(img_data), img_data.shape[1], img_data.shape[2], 3])
    for i in range(0, len(img_data)):
        im = 255 - (img_data[i] * 255)  # Orginal imagnet images were not rescaled
        im = color.gray2rgb(im)
        images[i] = im
    return(images)

### VGG_16 Bottleneck
The following function leverages Daniel's image loader function and performs the following:
1. Loads in the images using the train, test, and validation csv files.
2. Prepares the images using the VGG_Prep function
3. Loads the VGG_16 model with the cassification layers removed.
4. Runs each of the images for the training, test, and validation sets (if included) through the model.
5. Saves out .npy files containing the bottleneck features from the VGG_16 model predictions and the corresponding labels.

In [20]:
def vgg16_bottleneck(trainPath, testPath, imagePath, modelPath, size, balance = True, verbose = True, 
                     verboseFreq = 50, valPath = 'None', transform = False):
    # Loading data
    metaTr, metaTr2, mCountsTr = bc.load_training_metadata(trainPath, balance, verbose)
    lenTrain = len(metaTr)
    X_train, Y_train = bc.load_data(trainPath, imagePath, maxData = lenTrain, verboseFreq = verboseFreq, imgResize=size)
    
    metaTest, meataT2, mCountsT = bc.load_training_metadata(testPath, balance, verbose)
    lenTest = len(metaTest)
    X_test, Y_test = bc.load_data(testPath, imagePath, maxData = lenTrain, verboseFreq = verboseFreq, imgResize=size)
    
    if transform:
        print('Transforming the Training Data')
        X_train, Y_train = Image_Augment(X=X_train, Y=Y_train, hflip=True, vflip=True, minor_rotate=True, major_rotate=True)
    
    print('Preparing the Training Data for the VGG_16 Model.')
    X_train = VGG_Prep(X_train)
    print('Preparing the Test Data for the VGG_16 Model')
    X_test = VGG_Prep(X_test)
        
    print('Loading the VGG_16 Model')
    model = applications.VGG16(include_top=False, weights='imagenet')
        
    # Generating the bottleneck features for the training data
    print('Evaluating the VGG_16 Model on the Training Data')
    bottleneck_features_train = model.predict(X_train)
    
    # Saving the bottleneck features for the training data
    featuresTrain = os.path.join(modelPath, 'bottleneck_features_train.npy')
    labelsTrain = os.path.join(modelPath, 'labels_train.npy')
    print('Saving the Training Data Bottleneck Features.')
    np.save(open(featuresTrain, 'wb'), bottleneck_features_train)
    np.save(open(labelsTrain, 'wb'), Y_train)

    # Generating the bottleneck features for the test data
    print('Evaluating the VGG_16 Model on the Test Data')
    bottleneck_features_test = model.predict(X_test)
    
    # Saving the bottleneck features for the test data
    featuresTest = os.path.join(modelPath, 'bottleneck_features_test.npy')
    labelsTest = os.path.join(modelPath, 'labels_test.npy')
    print('Saving the Test Data Bottleneck Feaures.')
    np.save(open(featuresTest, 'wb'), bottleneck_features_test)
    np.save(open(labelsTest, 'wb'), Y_test)
    
    if valPath != 'None':
        metaVal, metaV2, mCountsV = bc.load_training_metadata(valPath, verbose = verbose, balanceViaRemoval = False)
        lenVal = len(metaVal)
        X_val, Y_val = bc.load_data(valPath, imagePath, maxData = lenVal, verboseFreq = verboseFreq, imgResize=size)
        X_val = VGG_Prep(X_val)
        
        # Generating the bottleneck features for the test data
        print('Evaluating the VGG_16 Model on the Validataion Data')
        bottleneck_features_val = model.predict(X_val)
    
        # Saving the bottleneck features for the test data
        featuresVal = os.path.join(modelPath, 'bottleneck_features_validation.npy')
        labelsVal = os.path.join(modelPath, 'labels_validation.npy')
        print('Saving the Validation Data Bottleneck Features.')
        np.save(open(featuresVal, 'wb'), bottleneck_features_val)
        np.save(open(labelsVal, 'wb'), Y_val)

## Running the model on the Train, Test, and Validation Data
1) The first test is on the rescaled and squared off images maintaining aspect ratio without the artifacts removed.

In [18]:
# global variables for loading the data
imagePath = '../images/ddsm/png/'
trainDataPath = '../images/ddsm/ddsm_train.csv'
testDataPath = '../images/ddsm/ddsm_test.csv'
valDataPath = '../images/ddsm/ddsm_val.csv'
imgResize = (224, 224) # can go up to (224, 224)
modelPath = '../model/'

In [21]:
vgg16_bottleneck(trainDataPath, testDataPath, imagePath, modelPath, imgResize, 
                 balance = True, verbose = True, verboseFreq = 50, valPath = valDataPath, 
                 transform = False)

Raw Balance
----------------
benign 531
malignant 739
normal 2685
balanaceViaRemoval.avgE: 1318
balanaceViaRemoval.theshold: 1318.0

After Balancing
----------------
benign 531
malignant 739
normal 862
Raw Balance
----------------
benign 531
malignant 739
normal 2685
balanaceViaRemoval.avgE: 1318
balanaceViaRemoval.theshold: 1318.0

After Balancing
----------------
benign 531
malignant 739
normal 862
0.0000: C_0418_1.LEFT_CC.LJPEG.png
0.0235: C_0325_1.LEFT_MLO.LJPEG.png
0.0469: B_3401_1.LEFT_CC.LJPEG.png
0.0704: C_0288_1.LEFT_MLO.LJPEG.png
0.0938: A_1097_1.LEFT_CC.LJPEG.png
0.1173: B_3362_1.RIGHT_CC.LJPEG.png
0.1407: B_3037_1.LEFT_MLO.LJPEG.png
0.1642: B_3399_1.RIGHT_CC.LJPEG.png
0.1876: B_3031_1.LEFT_CC.LJPEG.png
0.2111: A_0263_1.LEFT_MLO.LJPEG.png
0.2345: C_0400_1.RIGHT_MLO.LJPEG.png
0.2580: B_3603_1.RIGHT_MLO.LJPEG.png
0.2814: B_3154_1.LEFT_CC.LJPEG.png
0.3049: C_0072_1.LEFT_CC.LJPEG.png
0.3283: C_0136_1.LEFT_MLO.LJPEG.png
0.3518: B_3073_1.LEFT_MLO.LJPEG.png
0.3752: B_3062_1.LEFT_CC

In [11]:
class LossHistory(cb.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        batch_loss = logs.get('loss')
        self.losses.append(batch_loss)

## Train Top Model
This function takes the bottleneck features from the bottleneck function and applies a shallow CNN to these features to classify the images. The function needs to be pointed at the locations of the training and test features along with the training and test labels. You can use the epoch and batch size variables to control the number of images to show to the model and the number of training epochs. The model save variabler alows for saving of the final model weights.

In [12]:
def train_top_model(train_feats, train_lab, test_feats, test_lab, model_path, model_save, epoch = 50, batch = 64):
    train_bottleneck = os.path.join(model_path, train_feats)
    train_labels = os.path.join(model_path, train_lab)
    test_bottleneck = os.path.join(model_path, test_feats)
    test_labels = os.path.join(model_path, test_lab)
    
    history = LossHistory()
    
    X_train = np.load(train_bottleneck)
    Y_train = np.load(train_labels)
    Y_train = np_utils.to_categorical(Y_train, nb_classes=3)
    
    X_test = np.load(test_bottleneck)
    Y_test = np.load(test_labels)
    Y_test = np_utils.to_categorical(Y_test, nb_classes=3)

    model = Sequential()
    model.add(Flatten(input_shape=X_train.shape[1:]))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    # try Adadelta and Adam
    model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              nb_epoch=epoch,
              batch_size=batch,
              callbacks=[history],
              validation_data=(X_test, Y_test),
              verbose=2)
    
    score = model.evaluate(X_test, Y_test, batch_size=16, verbose=0)

    print "Network's test score [loss, accuracy]: {0}".format(score)
    
    model.save_weights(model_save)

## Running the Top Model
The following runs the top model classifier on the bottleneck features.

In [13]:
# Locations for the bottleneck and labels files that we need
modelPath = '../model/'
train_bottleneck = 'bottleneck_features_train.npy'
train_labels = 'labels_train.npy'
test_bottleneck = 'bottleneck_features_test.npy'
test_labels = 'labels_test.npy'
validation_bottleneck = 'bottleneck_features_valdation.npy'
validation_label = 'labels_validation.npy'
top_model_weights_path = 'top_weights02.h5'

In [10]:
train_top_model(train_feats=train_bottleneck, train_lab=train_labels, test_feats=test_bottleneck, test_lab=test_labels,
                model_path=modelPath, model_save=top_model_weights_path)

Train on 2132 samples, validate on 536 samples
Epoch 1/50
0s - loss: 6.5851 - acc: 0.4508 - val_loss: 5.6644 - val_acc: 0.4571
Epoch 2/50
0s - loss: 3.4335 - acc: 0.5563 - val_loss: 2.2755 - val_acc: 0.5000
Epoch 3/50
0s - loss: 1.0971 - acc: 0.6393 - val_loss: 1.2428 - val_acc: 0.5243
Epoch 4/50
0s - loss: 0.6560 - acc: 0.7317 - val_loss: 1.3331 - val_acc: 0.4963
Epoch 5/50
0s - loss: 0.5417 - acc: 0.7767 - val_loss: 1.3829 - val_acc: 0.5019
Epoch 6/50
0s - loss: 0.4130 - acc: 0.8354 - val_loss: 1.4304 - val_acc: 0.5392
Epoch 7/50
0s - loss: 0.3163 - acc: 0.8752 - val_loss: 1.5420 - val_acc: 0.5187
Epoch 8/50
0s - loss: 0.2728 - acc: 0.8963 - val_loss: 1.5069 - val_acc: 0.5168
Epoch 9/50
0s - loss: 0.2050 - acc: 0.9282 - val_loss: 1.6816 - val_acc: 0.5243
Epoch 10/50
0s - loss: 0.1788 - acc: 0.9273 - val_loss: 1.6940 - val_acc: 0.5485
Epoch 11/50
0s - loss: 0.1626 - acc: 0.9465 - val_loss: 1.7204 - val_acc: 0.5299
Epoch 12/50
0s - loss: 0.1236 - acc: 0.9578 - val_loss: 1.7611 - val_ac

## Running the Top Model on the Fully Augmented Data
In this run we will be using the bottleneck features from taking the training data and augmenting it with the followin transformations; Vertical Flip, Horizontal Flip, 90 and 270 degree rotation, and 15, 30, and 45 degree rotation in both directions.

In [16]:
# Locations for the bottleneck and labels files that we need
modelPath = '../model/'
train_bottleneck = 'bottleneck_features_150fulltrans_train.npy'
train_labels = 'labels_150fulltrans_train.npy'
test_bottleneck = 'bottleneck_features_test.npy'
test_labels = 'labels_test.npy'
validation_bottleneck = 'bottleneck_features_valdation.npy'
validation_label = 'labels_validation.npy'
top_model_weights_path = 'top_weights_150fulltrans.h5'

In [17]:
train_top_model(train_feats=train_bottleneck, train_lab=train_labels, test_feats=test_bottleneck, test_lab=test_labels,
                model_path=modelPath, model_save=top_model_weights_path)

Train on 23452 samples, validate on 536 samples
Epoch 1/50
9s - loss: 2.1150 - acc: 0.5010 - val_loss: 0.9707 - val_acc: 0.5429
Epoch 2/50
9s - loss: 0.8477 - acc: 0.6011 - val_loss: 1.0201 - val_acc: 0.5690
Epoch 3/50
9s - loss: 0.7545 - acc: 0.6495 - val_loss: 1.0338 - val_acc: 0.5709
Epoch 4/50
10s - loss: 0.6726 - acc: 0.6937 - val_loss: 1.1553 - val_acc: 0.5634
Epoch 5/50
12s - loss: 0.6003 - acc: 0.7329 - val_loss: 1.1349 - val_acc: 0.5709
Epoch 6/50
13s - loss: 0.5381 - acc: 0.7633 - val_loss: 1.2890 - val_acc: 0.5784
Epoch 7/50
13s - loss: 0.4795 - acc: 0.7914 - val_loss: 1.3350 - val_acc: 0.5728
Epoch 8/50
13s - loss: 0.4376 - acc: 0.8142 - val_loss: 1.3227 - val_acc: 0.5877
Epoch 9/50
12s - loss: 0.3839 - acc: 0.8383 - val_loss: 1.4640 - val_acc: 0.5560
Epoch 10/50
12s - loss: 0.3497 - acc: 0.8528 - val_loss: 1.5536 - val_acc: 0.5765
Epoch 11/50
12s - loss: 0.3168 - acc: 0.8711 - val_loss: 1.6186 - val_acc: 0.5784
Epoch 12/50
12s - loss: 0.2859 - acc: 0.8848 - val_loss: 1.624

## Running the Top Model at 224x224
In this next experiment we run the model with transformations on the data at a size of 224x224

In [22]:
# Locations for the bottleneck and labels files that we need
modelPath = '../model/'
train_bottleneck = 'bottleneck_features_train_224.npy'
train_labels = 'labels_train_224.npy'
test_bottleneck = 'bottleneck_features_test.npy'
test_labels = 'labels_test.npy'
validation_bottleneck = 'bottleneck_features_valdation.npy'
validation_label = 'labels_validation.npy'
top_model_weights_path = 'top_weights_224.h5'

In [23]:
train_top_model(train_feats=train_bottleneck, train_lab=train_labels, test_feats=test_bottleneck, test_lab=test_labels,
                model_path=modelPath, model_save=top_model_weights_path)

Train on 2132 samples, validate on 536 samples
Epoch 1/50
2s - loss: 9.0933 - acc: 0.4170 - val_loss: 8.1122 - val_acc: 0.4608
Epoch 2/50
2s - loss: 8.1198 - acc: 0.4634 - val_loss: 9.5227 - val_acc: 0.4030
Epoch 3/50
2s - loss: 7.4928 - acc: 0.5033 - val_loss: 7.4591 - val_acc: 0.4776
Epoch 4/50
2s - loss: 6.3570 - acc: 0.5432 - val_loss: 6.4214 - val_acc: 0.5466
Epoch 5/50
2s - loss: 5.6562 - acc: 0.5943 - val_loss: 5.6958 - val_acc: 0.5634
Epoch 6/50
2s - loss: 4.9501 - acc: 0.6196 - val_loss: 6.8575 - val_acc: 0.5168
Epoch 7/50
2s - loss: 4.3462 - acc: 0.6384 - val_loss: 7.0712 - val_acc: 0.4571
Epoch 8/50
2s - loss: 2.2618 - acc: 0.6731 - val_loss: 1.3050 - val_acc: 0.5914
Epoch 9/50
2s - loss: 0.7706 - acc: 0.7233 - val_loss: 1.1151 - val_acc: 0.5765
Epoch 10/50
2s - loss: 0.5902 - acc: 0.7617 - val_loss: 1.1186 - val_acc: 0.6101
Epoch 11/50
2s - loss: 0.4783 - acc: 0.8096 - val_loss: 1.2259 - val_acc: 0.6194
Epoch 12/50
2s - loss: 0.3681 - acc: 0.8462 - val_loss: 1.3938 - val_ac

## Results
All results below are run against the train, test, validate csv files located at [Breast Cancer Github Data](https://github.com/jnarhan/Breast_Cancer/tree/master/data)

### Aspect Ratio Squared Raw DDSM Images with Artifacts
1) Run 1: 150x150 image size, 50 Epochs, Batch Size 64
    * Network's test score [loss, accuracy]: [2.4609192387381595, 0.58582089552238803]
2) Run 2: 150x150 image size, 50 Epochs, Batch Size 64, Full Augmentations
    * Network's test score [loss, accuracy]: [2.8447668943832172, 0.57276119402985071]
3) Run 3: 224x224 image size, 50 Epochs, Batch Size 64
    * Network's test score [loss, accuracy]: [2.7020884663311402, 0.59888059701492535]